# FEATURISATION

In this Notebook we show how all of the different input features we're calculated

This includes:
1. Morgan Fingerprints
2. RDkit descriptors
3. Mordred descriptors
4. Multiple versions of MolBert molecular encoding
5. Latent Space Vectors of the JTNN model

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#### Loading in all the Smiles of the photoswitch dataset first

In [2]:
# Import main data and get list of SMILES
photoswitches = pd.read_csv('./photoswitches.csv')  # Load the photoswitch dataset using pandas
smiles_list = photoswitches['SMILES'].to_list()
len(smiles_list)

In [4]:
# Initiate list of rdkit molecules
rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]

# Get Morgan fingerprints, note the parameters!
morgan_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048) for mol in rdkit_mols]
morgan_fingerprints = np.asarray(morgan_fingerprints)

# Turn into pandas dataframe and add smiles as a first column
morgan_fingerprints = pd.DataFrame(data = morgan_fingerprints)
morgan_fingerprints.insert(0, "SMILES", smiles_list)

morgan_fingerprints

In [8]:
morgan_fingerprints.to_csv("morgan_fingerprints.csv")

## RDKIT Descriptors

In [1]:
# Next, rdkit's own descriptors
from rdkit.Chem import Descriptors

# A list of desriptors
with open("descriptors_rdkit.txt", "w") as f: 
  f.write(str(Descriptors.descList))
  
# Write a dictionary of name:function pairs for all descriptors
all_descriptors = {d[0]: d[1] for d in Descriptors.descList}
# Initialise a new pandas df
rdkit_descriptors = pd.DataFrame(data = {"SMILES": np.array((smiles_list)) })
rdkit_descriptors

In [13]:
# Compute each descriptor (outer loop) for each molecule(inside)
for feature in all_descriptors:
    values = []
    for mol in rdkit_mols:
        values += [all_descriptors[feature](mol)]
    rdkit_descriptors[feature] = values

rdkit_descriptors

,SMILES,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,C[N]1N=NC(=N1)N=NC2=CC=CC=C2,3.936481,0.260512,3.936481,0.260512,0.672275,188.194,180.130,188.081044,70,...,0,0,0,0,1,0,0,0,0,0
1,C[N]1C=NC(=N1)N=NC2=CC=CC=C2,3.976481,0.371623,3.976481,0.371623,0.676690,187.206,178.134,187.085795,70,...,0,0,0,0,0,0,0,0,0,0
2,C[N]1C=CC(=N1)N=NC2=CC=CC=C2,4.081997,0.621623,4.081997,0.621623,0.664734,186.218,176.138,186.090546,70,...,0,0,0,0,0,0,0,0,0,0
3,C[N]1C=C(C)C(=N1)N=NC2=CC=CC=C2,4.181534,0.667920,4.181534,0.667920,0.686454,200.245,188.149,200.106196,76,...,0,0,0,0,0,0,0,0,0,0
4,C[N]1C=C(C=N1)N=NC2=CC=CC=C2,4.061481,0.760512,4.061481,0.760512,0.664734,186.218,176.138,186.090546,70,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,OC%38=C%39N=CC=CC%39=C(/N=N/C%40=NC%41=CC(C)=C...,9.883085,0.134711,9.883085,0.134711,0.486213,334.404,320.292,334.088832,118,...,0,0,0,0,0,1,0,0,0,0
401,OC%42=C%43N=CC=CC%43=C(/N=N/C%44=NC%45=CC=CC=C...,9.822888,0.123414,9.822888,0.123414,0.541174,289.298,278.210,289.096360,106,...,0,0,0,0,0,0,0,0,0,0
402,N#CC1C(SC(/N=N/C2=NC(C=CC([N+]([O-])=O)=C3)=C3...,10.934342,-0.422909,10.934342,0.033505,0.424569,416.532,396.372,416.108916,146,...,1,0,0,0,0,1,0,0,0,0
403,N#Cc5c(c6ccc(Cl)cc6)c(/N=N/C7=NC(C=CC([N+]([O-...,10.920834,-0.461717,10.920834,0.014485,0.220871,440.897,431.825,439.991693,142,...,0,0,0,0,0,1,0,1,0,0


In [14]:
rdkit_descriptors.to_csv("rdkit_descriptors.csv")

## Mordred Descriptors

In [15]:
from mordred import Calculator, descriptors, error
calc = Calculator(descriptors)
len(calc.descriptors)

In [18]:
mordred_descriptors = calc.pandas(rdkit_mols)

 16%|█▌        | 63/405 [00:06<01:03,  5.39it/s]

/home/florian/anaconda3/envs/photoswitch/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 405/405 [00:27<00:00, 14.80it/s]


In [19]:
# It seems that unfortunately some descriptors cannot be computed. To filter this, 
# we find all columns that are of data type "object", since those contain non-numerical values usually.
error_columns = []
for i, e in enumerate(mordred_descriptors.dtypes):
    if e=="object":
        error_columns += [i]
error_columns

[780,
 781,
 782,
 783,
 784,
 785,
 786,
 787,
 788,
 789,
 790,
 791,
 792,
 793,
 794,
 795,
 796,
 797,
 798,
 799,
 800,
 801,
 802,
 803,
 804,
 805,
 806,
 807,
 808,
 809,
 810,
 811,
 812,
 813,
 814,
 817,
 818,
 819,
 820,
 821,
 822,
 906,
 907,
 908,
 909,
 910,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 919,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1104,
 1105,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1141,
 1142,
 1143,
 1144,
 1145,
 1146,
 1147,
 1148,
 1149,
 1150,
 1151,
 1152,
 1153,
 1154,
 1155,
 1156,
 1157,
 1158,
 1159,
 1160,
 1161,
 1162,
 1163,
 1164,
 1165,
 1166,
 1167,
 1168,
 1169,
 1170,
 1171,
 1172,
 1173,
 1174,
 1175,
 1176,
 1177,
 1178,
 1179,
 1180,
 1181,
 1182,
 1183,
 1184,
 1186,
 1187,
 1188

In [20]:
# use .drop to remove the affected columns 
mordred_descriptors = mordred_descriptors.drop(mordred_descriptors.columns[error_columns], axis=1)
# and remove columns containing NA data, but I don't think this actually does anything...
mordred_descriptors = mordred_descriptors.dropna(axis=1)
# again, insert first SMILES column
mordred_descriptors.insert(0, "SMILES", smiles_list)
mordred_descriptors

In [23]:
mordred_descriptors.to_csv("mordred_descriptors.csv")

## MOLBERT Features

In [ ]:
# first of all clone the MolBERT github repository
!git clone https://github.com/BenevolentAI/MolBERT.git

# then create this conda env
!conda create -y -q -n molbert -c rdkit rdkit=2019.03.1.0 python=3.7.3

# go to the molbert folder, activate the env, and install the package
!cd MolBERT
!conda activate molbert
!pip install -e .

At this point, it might be wise to restart the jupyter notebook with the newly created molbert conda env as a kernel, so that you have access to the molbert package

In [ ]:
from molbert.utils.featurizer.molbert_featurizer import MolBertFeaturizer

path_to_checkpoint = '.MolBERT/molbert_100epochs/checkpoints/last.ckpt'
f = MolBertFeaturizer(path_to_checkpoint, device='cpu')

smiles = pd.read_csv('./raw_data/photoswitches.csv').to_numpy()[:, 1]

features, masks = f.transform(smiles)
assert all(masks)

index = ['Row'+str(i) for i in range(1, len(features[0]) + 1)]
df = pd.DataFrame(features, columns=index)
df.insert(loc=0, column='Smiles', value=smiles)

df.to_csv('./processed_data/molbert_features.csv')

Now this model is simply trained with a pretrained MolBERT model. However in order to increase the accuracy, we might train to finetune the MolBERT model using our photoswitch dataset

In [ ]:
# Here we split our photoswitch dataset in a train, test & validation set
df = pd.read_csv('./raw_data/photoswitches.csv')
columns = df.columns

name = 'E isomer pi-pi* wavelength in nm'
property_vals = df['E isomer pi-pi* wavelength in nm'].to_numpy()
invalid_indices = np.argwhere(np.isnan(property_vals))

df = np.delete(df.to_numpy(), invalid_indices, axis=0)

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.25, random_state=1)

pd.DataFrame(df_train, columns=columns).to_csv('./MolBERT/df_train.csv')
pd.DataFrame(df_test, columns=columns).to_csv('./MolBERT/df_test.csv')
pd.DataFrame(df_val, columns=columns).to_csv('./MolBERT/df_val.csv')

In [ ]:
# these command lines can be used to train a finetuned MolBERT model - we did this on a cloud EC2 instance
!cd MolBERT
!python3 ./molbert/apps/finetune.py --train_file ./df_train.csv --valid_file ./df_val.csv --test_file ./df_test.csv --mode regression --output_size 1 --pretrained_model_path ./molbert_100epochs/checkpoints/last.ckpt --label_column "E isomer pi-pi* wavelength in nm" --gpus 1 --freeze_level 0

In [ ]:
# when the training is done

# replace this path with the path where your checkpoint was saved
path_to_checkpoint = 'C:/users/rhjva/version_7/checkpoints/last.ckpt'

f = MolBertFeaturizer(path_to_checkpoint, device='cpu')

smiles = pd.read_csv('../raw_data/photoswitches.csv').to_numpy()[:, 1]

features, masks = f.transform(smiles)
assert all(masks)

index = ['Row'+str(i) for i in range(1, len(features[0]) + 1)]
df = pd.DataFrame(features, columns=index)
df.insert(loc=0, column='Smiles', value=smiles)

df.to_csv('../processed_data/molbert_tuned_features.csv')

We can play with the level of freezing the weights of the MolBERT models, which is what we did as well:

freeze level -> File <br>
 0 -> molbert_tuned_features.csv  <br>
-1 -> molbert_tuned_2_features.csv <br>
-4 -> molbert_tuned_3_features.csv <br>
-2 -> molbert_tuned_4_features.csv <br>
-3 -> molbert_tuned_5_features.csv <br>

## JTNN Features

In [1]:
# run this command to clone the JTNN repository
!git clone https://github.com/Bibyutatsu/FastJTNNpy3.git

# now run: cd FastJTNNpy3
# and make a file called: __init__.py (this can stay empty)

fatal: destination path 'FastJTNNpy3' already exists and is not an empty directory.


In [ ]:
from FastJTNNpy3.fast_jtnn import *

import pandas as pd
import torch
torch.cuda.empty_cache()

df = pd.read_csv('./raw_data/photoswitches_2.csv')

def generate_features(vocab, model_path, hidden_size=450, latent_size=56, depthT=20, depthG=3):
  vocab = [x.strip("\r\n ") for x in open(vocab)] 
  vocab = Vocab(vocab)

  model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG)
  dict_buffer = torch.load(model_path)
  model.load_state_dict(dict_buffer)
  model = model.cpu()

  torch.manual_seed(0)
  
  features = model.encode_from_smiles(df['SMILES'])
  
  index = ['Row'+str(i) for i in range(1, len(features[0]) + 1)]
  df = pd.DataFrame(features, columns=index)
  df.insert(loc=0, column='Smiles', value=smiles)

  df.to_csv('./processed_data/jtnn_features.csv')
  
vocab = './FastJTNNpy3/data/vocab.txt'
generate_features(vocab, './FastJTNNpy3/fast_molvae/vae_model/model.epoch-19')